<a href="https://colab.research.google.com/github/schuhf/schuhf/blob/main/gif_usodosolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GIF com transição do uso de solo entre 1985 e 2021 usando dados do MAPBIOMAS

Autor: Fernando Schuh Rörig

Engenheiro Ambiental

Doutorando em Geociências na USP

Linkedin: https://www.linkedin.com/in/fernando-schuh-r%C3%B6rig-a634ba168/


## 1. Inicializando

In [ ]:
## instalando pacotes necessários ao colab (não nativos)
!pip install geemap
!pip install geopandas
!pip install plotnine
!pip install mizani
!pip install datar
!pip install imageio
!pip install fiona
!pip install rasterio
!pip install gdal
!pip install opencv-python

In [2]:
## carregando os pacotes (depois de terminar o código, gosto de juntar tudo aqui para deixar mais limpo)

## pacotes para abrir arquivos
from google.colab import drive
import os
import glob

## pacotes para análise geoespacial
import ee
import geemap
import gdal
import geopandas

## pacotes para trabalhar com os dados
import pandas as pd
import numpy as np

## pacotes para visualização de dados
from plotnine import *
from mizani.formatters import percent_format
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec

## pacotes para trabalhar com imagens
import imageio
import cv2

In [ ]:
# Autenticar a plataforma Google Earth Engine (GEE)
ee.Authenticate()

# Inicializar o GEE
ee.Initialize()

In [4]:
# Conectar com o google drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [5]:
## Abrir o shapefile da área de estudo
area_estudo = geemap.shp_to_ee('/content/drive/MyDrive/SACRE/SIG/Area Estudo/Area_Estudo_Ret.shp')

In [6]:
## Criando lista dos anos de interesse
anos_list = list(range(1985,2022)) ## lista de anos: 1985 = ano inicial e 2022 = ano final + 1 (vamos usar essa lista em mais iterações futuras)

class_anos_list = ['classification_' + str(ano) for ano in anos_list] ## lista para seleção

lista_ee_images = [] ## lista vazia para iterar

### Abrindo dados da Coleção 7.0 do MAPBIOMAS (mais informações em: https://mapbiomas.org/produtos)
for ano_class in class_anos_list:
  image = ee.Image('projects/mapbiomas-workspace/public/collection7/mapbiomas_collection70_integration_v2').select(ano_class)
  lista_ee_images.append(image)

## 2. Estatística zonal

Aqui vamos calcular a área estimada para cada classe de uso do solo da Coleção 7.0 do MAPBIOMAS (a legenda completa e a descrição de cada classe está disponível em: https://mapbiomas.org/codigos-de-legenda)

In [ ]:
## Extraindo os dados pelo shape

## selecionando pasta para salvar estatísticas
out_dir = os.path.expanduser('/content/drive/MyDrive/SACRE/SIG/Landcover')

aoi_stats_paths = ['aoi' + str(ano) + 'lc_stats.csv' for ano in anos_list]

## calculando estatísticas para cada ano
for image, path in zip(lista_ee_images, aoi_stats_paths):
  aoi_stats = os.path.join(out_dir, path)

  ## garantindo que diretório de exportação irá existir
  if not os.path.exists(out_dir):
      os.makedirs(out_dir)
      
  ### Calculando a área em km²
  df = geemap.zonal_statistics_by_group(image, area_estudo, aoi_stats, statistics_type = 'SUM', denominator = 1000000, decimal_places = 2)

In [9]:
### Inserção do nome da classe
dicionario_classes = {
    0:"Fora da área de interesse",
    1:"Floresta",
    3:"Formação Florestal",
    4:"Formação Savânica",
    5:"Mangue",
    49:"Restinga Arborizada",
    10:"Formação Natural não Florestal",
    11:"Campo Alagado e Área Pantanosa",
    12:"Formação Campestre",
    32:"Apicum",
    29:"Afloramento Rochoso",
    50:"Restinga Herbácea",
    13:"Outras Formações não Florestais",
    14:"Agropecuária",
    15:"Pastagem",
    18:"Agricultura",
    19:"Lavoura Temporária",
    39:"Soja",
    20:"Cana",
    40:"Arroz (beta)",
    62:"Algodão (beta)",
    41:"Outras Lavouras Temporárias",
    36:"Lavoura Perene",
    46:"Café",
    47:"Citrus",
    48:"Outras Lavouras Perenes",
    9:"Silvicultura",
    21:"Mosaico de Usos",
    22:"Área não Vegetada",
    23:"Praia, Duna e Areal",
    24:"Área Urbanizada",
    30:"Mineração",
    25:"Outras Áreas não Vegetadas",
    26:"Corpo D'água",
    33:"Rio, Lago e Oceano",
    31:"Aquicultura",
    27:"Não observado"
}

### Inserção das cores de legenda
dicionario_cores = {
    0:"white",
    1:"#129912",
    3:"#006400",
    4:"#00ff00",
    5:"#687537",
    49:"#6b9932",
    10:"#bbfcac",
    11:"#45c2a5",
    12:"#b8af4f",
    32:"#968c46",
    29:"#ff8C00",
    50:"#66ffcc",
    13:"#bdb76b",
    14:"#ffffb2",
    15:"#ffd966",
    18:"#e974ed",
    19:"#d5a6bd",
    39:"#c59ff4",
    20:"#c27ba0",
    40:"#982c9e",
    62:"#660066",
    41:"#e787f8",
    36:"#f3b4f1",
    46:"#cca0d4",
    47:"#d082de",
    48:"#cd49e4",
    9:"#935132",
    21:"#fff3bf",
    22:"#ea9999",
    23:"#dd7e6b",
    24:"#af2a2a",
    30:"#8a2be2",
    25:"#ff99ff",
    26:"#0000ff",
    33:"#0000ff",
    31:"#29eee4",
    27:"#D5D5E5"
}

## paleta
paleta_nomes = {key:value for key, value in zip(dicionario_classes.values(), dicionario_cores.values())}

In [ ]:
## Abrindo os dfs das estatísticas zonais
path = '/content/drive/MyDrive/SACRE/SIG/Landcover'

dfs_files = glob.glob(os.path.join(path, '*.csv'))

df_list = [] ## lista vazia para iterar

for file_path, ano in zip(dfs_files, anos_list):

    df = pd.read_csv(file_path, encoding = 'latin-1') ## carrega o dataframe

    ## ajustando colunas do df
    df = df.drop('Class_sum', axis = 1)
    filter_col_class = [col for col in df if col.startswith('Class')]
    df = df[filter_col_class]

    ## criando coluna de classe e de área
    df = df.melt(var_name="classe", value_name="area_km2")
    df['perc_area'] = df['area_km2'] / df['area_km2'].sum()
    df['ano'] = ano

    ## criando coluna com nome das classes de uso do solo
    df['classe'] = df['classe'].apply(lambda x: x.replace('Class_','')).astype(int) #limpando a coluna de classe
    df['nome_classe'] = df['classe']
    df['nome_classe'].replace(dicionario_classes, inplace = True)
    df = df.sort_values(by='perc_area', ascending=False)

    ## adiciona o df limpo à lista
    df_list.append(df)

## cria um dataframe completo
df_completo = pd.concat(df_list, axis = 0)

## observa o formato do df completo
df_completo.head()

In [ ]:
## Plot geral - evolução do uso do solo
plot = (
    ggplot(df_completo, aes(x = 'ano', y = 'perc_area', color = 'reorder(nome_classe,perc_area, ascending = False)'))
    + geom_line()
    + scale_color_manual(values = paleta_nomes)
    + scale_y_continuous(labels = percent_format(), expand = (0,0,0,0))
    + labs(x = 'Anos', y = 'Percentual de área da região de interesse', color = 'Classe de uso do solo', title = f'Evolução do uso de solo na área de interesse entre {anos_list[0]} e {anos_list[-1]}\nColeção 7.0 - MAPBIOMAS')
    + theme_minimal()
    #+ facet_wrap('nome_classe', ncol = 1, scales = 'free_y')
    + theme(figure_size = (8,8), text = element_text(family = 'sans-serif'))
)
plot

In [ ]:
## Plot geral - evolução do uso do solo com visualização da tendência de cada grupo
plot = (
    ggplot(df_completo, aes(x = 'ano', y = 'perc_area', color = 'reorder(nome_classe,perc_area, ascending = False)'))
    + geom_line()
    + scale_color_manual(values = paleta_nomes)
    + scale_y_continuous(labels = percent_format(), expand = (0,0,0,0))
    + labs(x = 'Anos', y = 'Percentual de área da região de interesse (escala variável entre as classes!)', color = 'Classe de uso do solo', title = f'Evolução do uso de solo na área de interesse entre {anos_list[0]} e {anos_list[-1]}\nColeção 7.0 - MAPBIOMAS')
    + theme_bw()
    + facet_wrap('nome_classe', ncol = 1, scales = 'free_y')
    + theme(figure_size = (8,20), legend_position = 'none')
)
plot

In [65]:
%%capture

## a função %%capture faz com que não haja outputs sendo printados a cada exportação de gráfico

## plot de barras para cada ano

for df, ano in zip(df_list, anos_list):
  ## plotando com plotnine
  p = (
      ggplot(df, aes(y = 'perc_area', x = 'reorder(nome_classe, perc_area)', fill = 'nome_classe'))
      + coord_flip()
      + geom_bar(stat = 'identity')
      + scale_fill_manual(values = paleta_nomes)
      + scale_y_continuous(labels = percent_format(), expand = (0,0,0,0.05), breaks = (0,0.10,0.20,0.30,0.40,0.50), limits = (0,0.55))
      + geom_text(aes(label = 'perc_area*100'), size = 10, position = position_nudge(0,0.04),format_string="{:,.2f}%")
      + labs(x = '', y = 'Percentual de área (%)',  title = 'Classes de uso de solo na área de interesse em {}\nColeção 7.0 MAPBIOMAS'.format(ano))
      + theme_minimal()
      + theme(legend_position = 'none', figure_size = (5,4))
  )

  ## exportando figura 
  ggsave(plot=p, filename='/content/drive/MyDrive/SACRE/SIG/Landcover/PercentPlot/fig_percent_lc_{}.png'.format(ano), dpi=500)

## 3. Visualização da imagem de uso do solo

In [29]:
area_estudo = ee.FeatureCollection(area_estudo)

for im, ano in zip(lista_ee_images, anos_list):

  clip = im.clipToCollection(area_estudo)  # recorta a imagem com o limite da área de estudo

  ## exporta o resultado
  region = area_estudo.geometry()
  scale = 50
  folder = r'content//drive//MyDrive//SACRE//SIG//Landcover//Rasters'
  img_name = 'lc_img_{}'.format(ano)

  export_config = {'scale': scale, 'maxPixels': 1.0E13, 'driveFolder': folder, 'region': region}

  task = ee.batch.Export.image(clip, img_name, export_config)
  task.start()

In [32]:
## definindo função para recodificar o raster para a adequada visualização

def replace_with_dict(ar, dic):
    # Extrai chaves e valores
    k = np.array(list(dic.keys()))
    v = np.array(list(dic.values()))

    # Pega argsort indices
    sidx = k.argsort()
    
    ks = k[sidx]
    vs = v[sidx]
    return vs[np.searchsorted(ks,ar)]

In [55]:
%%capture 

## define caminho dos rasters
path = '/content/drive/MyDrive/SACRE/SIG/Landcover/Rasters'
raster_files = glob.glob(os.path.join(path, '*.tif'))
raster_files = sorted(raster_files)

## cria visualização para cada raster
for raster, ano in zip(raster_files, anos_list):
  ds = gdal.Open(raster)
  myarray = np.array(ds.GetRasterBand(1).ReadAsArray())

  ## recodifica valores
  vals = list(np.unique(myarray))
  recode_vals = list(range(1,len(vals) + 1))
  translate_vals = {val:recode for val, recode in zip(vals, recode_vals)}
  recoded_array = replace_with_dict(myarray, translate_vals)

  ## dicionario de cores para a paleta
  vals = list(np.unique(myarray))
  col_dict = {key: dicionario_cores[key] for key in vals}
  col_dict_recoded = {num:col for num, col in zip(recode_vals, list(col_dict.values()))}

  ## colormap a partir da lista de cores
  cm = ListedColormap([col_dict_recoded[x] for x in col_dict_recoded.keys()])

  ## descrição de cada categoria
  labels_dict = {key: dicionario_classes[key] for key in vals}
  labels = list(labels_dict.values())
  len_lab = len(labels)

  # normalizer
  ## prepara caixas para o normalizer
  norm_bins = np.sort([*col_dict_recoded.keys()]) + 0.5
  norm_bins = np.insert(norm_bins, 0, np.min(norm_bins) - 1.0)
  print(norm_bins)

  ## cria o normalizer e o formatter
  norm = matplotlib.colors.BoundaryNorm(norm_bins, len_lab, clip=True)
  fmt = matplotlib.ticker.FuncFormatter(lambda x, pos: labels[norm(x)])

  ## plotando
  fig, ax = plt.subplots()
  ax.set(xlabel=None, ylabel = None)
  ax.grid(False)
  ax.axis('off')
  fig.set_size_inches(11,7, forward = True)

  ## alinhando
  gs = GridSpec(1, 3, width_ratios=[1, 25, 1])
  plt_ax = fig.add_subplot(gs[1])
  plt_ax.set(xlabel=None, ylabel = None)
  im = plt_ax.imshow(recoded_array, cmap = cm, norm = norm) ## função de plotagem

  plt.title('Classes de uso do solo na área de estudo\nem {} (Coleção 7.0 MAPBIOMAS)'.format(ano), fontsize = 15) ## título
  plt.axis('off')
  plt.grid(False)

  ## legenda
  diff = norm_bins[1:] - norm_bins[:-1]
  tickz = norm_bins[:-1] + diff / 2
  cb = fig.colorbar(im, format=fmt, ticks=tickz)

  ## exportando
  fig.savefig("/content/drive/MyDrive/SACRE/SIG/Landcover/RastersPlot/landcover_fig_{}.png".format(ano), dpi = 500)

## 4. Montando o GIF

In [56]:
## definindo uma função para concatenar horizontalmente imagens de diferentes alturas

def hconcat_resize(img_list, 
                   interpolation 
                   = cv2.INTER_CUBIC):
      # pega altura mínima
    h_min = min(img.shape[0] for img in img_list)
      
    # formatando imagens
    im_list_resize = [cv2.resize(img,(int(img.shape[1] * h_min / img.shape[0]), h_min), interpolation = interpolation) for img in img_list]
      
    # retorna a imagem final concatenada
    return cv2.hconcat(im_list_resize)

In [66]:
## combina imagens
for ano in anos_list:
  img1 = cv2.imread('/content/drive/MyDrive/SACRE/SIG/Landcover/PercentPlot/fig_percent_lc_{}.png'.format(ano))
  img2 = cv2.imread('/content/drive/MyDrive/SACRE/SIG/Landcover/RastersPlot/landcover_fig_{}.png'.format(ano))

    # chama a função definida anterioremente
  img_h_resize = hconcat_resize([img2, img1])
      
    # salva o output
  cv2.imwrite('/content/drive/MyDrive/SACRE/SIG/Landcover/JoinPlot/join_plot_{}.png'.format(ano), img_h_resize)

In [67]:
## cria os frames do GIF
frames = []

for ano in anos_list:
  image = imageio.imread(f'/content/drive/MyDrive/SACRE/SIG/Landcover/JoinPlot/join_plot_{ano}.png')
  frames.append(image)

In [68]:
## combina os frames e gera o GIF
imageio.mimsave('/content/drive/MyDrive/SACRE/SIG/Landcover/gif_final.gif', # output gif
                frames,          # frames
                fps = 0.75)         # frames por segundo ("velocidade do gif")